## Imports

In [ ]:
from __future__ import annotations

from pathlib import Path

from kub.course.plotlib.simulationPlotFactory import SimulationPlotFactory
from kub.course.simlib.simulation import FMUSimulation

# Find repository root: check if database/ exists at current location
repo_root = Path.cwd() if (Path.cwd() / "database").exists() else Path.cwd().parent.parent
databasePath = repo_root / "database" / "day2"

# Thermal Building Optimization Exercise III: **Two-Zone Building with Gable Roof**

The building now consists of two thermal zones:
1.  **`zone_habitable_1` (Ground Floor):** The main zone, actively heated/cooled (as in Exercises I & II).

2.  **`zone_habitable_2` (Attic/Ceiling Space):** A new, **unheated/uncooled zone** .

The flat roof has been replaced by a **gable (two-slope) roof structure**. Your primary goal remains to minimize the total energy consumption of the **heated zone 1**, but this is now influenced by heat transfer through the slab separating the two zones and the unconditioned attic.

***

## Learning Objectives

Upon completion of this exercise, you should be able to:

1.  **Analyze Glazing Impact:** Quantify the trade-off between heat losses (conduction) and solar gains (radiation) through transparent elements.

2.  **Optimize Glazing Distribution:** Determine the optimal Window-to-Wall Ratio (WWR) distribution across façades based on building orientation and climate.

3.  **Integrate Design Decisions:** Assess how glazing properties (`UWindow`) and area interact with the building's thermal inertia (opaque envelope from Exercise I).

***

## 1. New Building Elements and Constraints

### Roof Geometry and Glazing

* **Gable Roof Inclination:** The roof consists of two panels whose inclination angle, **`IncRoof` (in degrees)**, can be modified.

    * **Shallow Roof (Low Angle):** Test values around **$10^\circ$ to $15^\circ$**. A shallow roof provides a smaller roof surface area exposed to the environment but can reduce heat gain via solar radiation in winter compared to a steep slope.

    * **Steep Roof (High Angle):** Test values around **$40^\circ$ to $50^\circ$**. A steep roof increases the surface area of the vertical gable walls and the roof panels themselves but can increase solar gain in winter.

* **Roof Windows:** Two new windows (`window_1_floor_2` and `window_2_floor_2`) are introduced on the roof panels.

* **Window Area Constraint:** The area of each window (both on the façade and on the roof) must be strictly **smaller** than the total area of the surface it is attached to.

### Inter-Zone Slab

* A **ceiling slab** (`slab_floor_2`) separates the heated zone (`zone_habitable_1`) from the unheated attic zone (`zone_habitable_2`).

* The properties of this slab (thickness: `eIntSlab`, `eExtSlab`, etc.) are important, as they determine the heat transfer between the two zones.

### Geometric and Opaque Material Constraints

* **All constraints from Exercise I remain active** (Area=$100\,\text{m}^2$, $2.7 \le height \le 3.0\,\text{m}$, $\text{Thickness} \le 0.40\,\text{m}$).

* For this exercise, use the **optimized opaque envelope design** (wall materials and orientation) determined in Exercise I as your starting point.

***

## 2. Assignment: Two-Zone Optimization Strategy

Your task is to find the overall optimal design by balancing the heat losses/gains from the exterior envelope (walls, roof, windows) with the heat exchange between the two zones via the slab.

### Suggested Investigation Steps:

1.  **Final Opaque and Glazing Baseline:**

    * Start with the **optimized façade design** (Wall thickness, orientation, and window sizes/U-value) determined in Exercises I and II.

    * Set an initial `IncRoof` (e.g., $30^\circ$).

2.  **Slab Optimization (Inter-Zone Barrier):**

    * The slab is important as it separates the conditioned zone (1) from the environment of the unheated attic (2).

    * **Insulation vs. Inertia:** Determine the optimal material properties for the slab (`eSlab`, $\lambda S\text{lab}$, $\rho S\text{lab}$) to minimize heat loss/gain from Zone 1. Should the slab prioritize **insulation** (low $\lambda$) or **thermal inertia** (high $\rho$)?

3.  **Roof Geometry and Window Optimization:**

    * **Vary `IncRoof`:** Investigate the impact of a shallow roof vs. a steep roof on total energy consumption.

    * **Roof Glazing:** Since Zone 2 is unheated, placing windows on the roof panels (Window 5 & 6) can affect the **temperature swing** of the attic space. Determine the optimal size and U-value for these roof windows.

* **Insulation vs. Inertia:** Determine the optimal material properties for the roof (`eRoof`, $\lambda S\text{lab}$, $\rho S\text{lab}$) to minimize heat loss/gain from Zone 1. Should the roof prioritize **insulation** (low $\lambda$) or **thermal inertia** (high $\rho$)? How does it correlate to slab material choice (Insulated roof with high inertia slab or both well insulated, or high inertia roof with well insulated slab)?

    * **Analysis:** Compare `building.zone_habitable_1.TAir` (heated) with `building.zone_habitable_2.TAir` (unheated) (Graph 1) to understand how the attic temperature acts as a thermal buffer (or a heat sink/source).

4.  **Final Synthesis:**

    * Propose the ultimate design combining your findings from all three exercises (Geometry, Opaque Walls, Façade Glazing, Slab, and Gable Roof).

### Success Criteria:

* The final configuration must adhere to **all geometric and material constraints** (including the new window area limits).

* The final configuration must **minimize `energyConsumptionTotal`** over the simulation period.


# Exercise resolution: ***TO DO***

### **Building thermal simulation**

In [2]:
inputs3 = {}

In [4]:
# variables to display
temperatureList1 = [
    "ambience.TAirRef",
    "building.zone_habitable_1.TAir",
    "building.zone_habitable_2.TAir",
]

energyList1 = [
    "energyConsumptionHeating",
    "energyConsumptionCooling",
    "energyConsumptionTotal",
]

In [5]:
# simulation
sim3 = FMUSimulation(
    databasePath
    / "BuildingModels"
    / "Exercices_Optimization_LoD0Bui4WallsGlazed1GableRoof_AppLoD0Bui4WallsGlazed1GableRoof.fmu"
)

sim3.initialize(startTime=0.0, stopTime=86400 * 365, timeStep=3600.0)
sim3.initParameters(inputs3)
sim3.exitInitialization()

data3 = sim3.run(temperatureList1 + energyList1)

sim3Factory = SimulationPlotFactory()